**Regression analysis Chbasani Apr 13,2020**
# NYC taxi data
# Only training

In [7]:
import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn.linear_model import LinearRegression

In [ ]:
#data = pd.read_json('https://data.cityofnewyork.us/resource/biws-g3hs.json')  # load data set
#X = data.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
#Y = data.iloc[:, 1].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
#linear_regressor = LinearRegression()  # create object for the class
#linear_regressor.fit(X, Y)  # perform linear regression
#Y_pred = linear_regressor.predict(X)  # make predictions

## Regression analysis Chbasani Apr 13,2020

NYC taxi data
training - test - validate


## Step 1 
Read the json data

In [9]:
nyctaxi = pd.read_json('https://data.cityofnewyork.us/resource/biws-g3hs.json')  # load data set
nyctaxi.head(5)

dolocationid  extra    ...     trip_distance  vendorid
0           237    0.0    ...              0.76         2
1           141    0.0    ...              0.60         1
2           170    0.0    ...              0.44         2
3           234    0.0    ...              1.54         2
4           164    0.0    ...              0.33         2

[5 rows x 17 columns]

## Step 2
Drop the columns


In [10]:
'''
{"vendorid":"2",
"tpep_pickup_datetime":"2017-12-01T07:04:55.000",
"tpep_dropoff_datetime":"2017-12-01T07:15:47.000",
"passenger_count":"1",
"trip_distance":"1.45",
"ratecodeid":"1",
"store_and_fwd_flag":"N",
"pulocationid":"164",
"dolocationid":"50",
"payment_type":"2",
"fare_amount":"9",
"extra":"0",
"mta_tax":"0.5",
"tip_amount":"0",
"tolls_amount":"0",
"improvement_surcharge":"0.3",
"total_amount":"9.8"}
'''

columns_to_remove = ["vendorid","tpep_pickup_datetime","tpep_dropoff_datetime",
"passenger_count","ratecodeid","store_and_fwd_flag","pulocationid","dolocationid","payment_type","fare_amount","extra","mta_tax",
"tip_amount","tolls_amount","improvement_surcharge"]
                  
for col in columns_to_remove:
    nyctaxi.pop(col)

nyctaxi.head(5)

total_amount  trip_distance
0          9.30           0.76
1          7.55           0.60
2          7.56           0.44
3          7.30           1.54
4          5.30           0.33

In [11]:
nyctaxi.describe()

total_amount  trip_distance
count   1000.000000    1000.000000
mean      16.794290       2.676580
std       14.177461       3.652278
min        0.000000       0.000000
25%        8.760000       0.900000
50%       12.350000       1.400000
75%       18.560000       2.500000
max      115.860000      23.800000

## Step 3 
Manipulation / Cleanup


In [12]:
#final_df = nyctaxi.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
#final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = nyctaxi.query("trip_distance>=0.25 and trip_distance<31")
final_df = final_df.query(" total_amount>0")



## Step 4
final dataset for modeling


In [13]:
final_df.describe()

total_amount  trip_distance
count    976.000000     976.000000
mean      17.018525       2.740359
std       14.143689       3.673804
min        3.800000       0.300000
25%        8.800000       0.900000
50%       12.530000       1.400000
75%       18.800000       2.505000
max      115.860000      23.800000

## Step 5 
Split dataset to train and test


In [14]:
from sklearn.model_selection import train_test_split

y_df = final_df.pop("total_amount")
x_df = final_df

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

## Step 6 
Linear Regression model on the training data. 
Predict using the model on the test data 


In [15]:
linear_regressor = LinearRegression()  # create object for the class
linear_regressor.fit(x_train, y_train)  # perform linear regression
y_predict = linear_regressor.predict(x_test)  # make predictions

## Step 7
Model Accuracy


In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

5.486396045068067

In [17]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.1885697743674144

Model Accuracy:
0.8114302256325856

## Step 8 
Visualization of model fit


In [18]:
plt.cla()   # Clear axis
plt.clf()   # Clear figure
plt.scatter(x_df, y_df)
plt.plot(x_test, y_predict, color='red')
plt.show()